# Analyse data for line loop navigation coming from the homecage 

In [15]:
%matplotlib inline

import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
import scipy.stats as stt
import pandas as pd

##  Main function to analyse probe trial performance

In [50]:
df = pd.DataFrame(columns=['subject','time','seq','graph_type','two_tasks','nRews','probe_dict'])

In [55]:
def get_experiment_name(line):
    return re.findall('Experiment name[\s] \: (.*)\n',line)[0]

def get_subject_name(line):
    return re.findall('Subject ID[\s]:[\s](.*)\n',line)[0]

def get_start_date(line):
    return re.findall('Start date[\s]:[\s](.*)\n',line)[0]

In [44]:
re.findall('Subject ID[\s]:[\s](.*)\n',lines[2])

['C00499495_6']

In [48]:
re.findall('Start date[\s]:[\s](.*)\n',lines[3])

['2021-05-11-081332']

In [82]:
def get_probe_trial_performance_homecage(lines,probe_dict={}):
    """ NEED TO UPDATE THIS TO INCLUDE TIME LIMIT"""
    times = []
    probe_perf = []
    target_hist = []; probe_hist = []; poked_hist = []; dir_hist = []; seq_hist = []
    #if probe_dict is None: 
    #    probe_dict = {}
    probe_dict_ = {}
    task_nums = []; seqs = [];
    info_dict = {}
    for ix,l_ in enumerate(lines[:-1]):
        
        if 'Subject ID' in l_: info_dict['subject'] = get_subject_name(l_)
        if 'Experiment name' in l_: info_dict['exp_name'] = get_experiment_name(l_)
        if 'Start date' in l_: info_dict['date'] = get_start_date(l_)
            
        
        if 'task_number' in l_:
            task_number = int(re.findall('task_number([0-9])',l_)[0])
            task_nums.append(task_number)
            
        if 'seq' in l_:
            seq = eval(re.findall('seq:(\[.*\])', 'P 1 seq:[3, 6, 1]\n',)[0])
            seqs.append(seq)
            
        if 'TARGET:' in l_:
            t_ = re.findall('P ([0-9]*) R',l_)[0]
            times.append(float(t_))
            probe = eval(re.findall('PROBE: (False|True)',l_)[0])
            target = int(re.findall('TARGET: ([0-9]*)',l_)[0])
            poked = int(re.findall('POKED: ([0-9]*)',l_)[0])
            direction = int(re.findall('DIR: (.*)',l_)[0])
            seq_ix = int(re.findall('SEQ_IX: ([0-9]*)',l_)[0])
            if seq_hist:
                if probe:
                    #print(target)
                    if not (probe_hist[-1] and (target==target_hist[-1])):
                        correct_poke = poked==target
                        probe_perf.append(correct_poke)
                        transition = str(target_hist[-1]) + '_' + str(target)
                        #print(probe_dict.keys())
                        if transition in probe_dict.keys():
                            probe_dict[transition].append(correct_poke)
                        else:
                            probe_dict[transition] = [correct_poke]
                            
                        if transition in probe_dict_.keys():
                            probe_dict_[transition].append(correct_poke)
                        else:
                            probe_dict_[transition] = [correct_poke]

            target_hist.append(target); probe_hist.append(probe); dir_hist.append(direction);
            poked_hist.append(poked); seq_hist.append(seq_ix)
    #print(len(probe_perf))
    info_dict['seqs'] = seqs; info_dict['task_nr'] = task_nums; info_dict['probe_dict'] = probe_dict_
    info_dict['len_seq'] = [len(i) for i in seqs]
    return probe_dict,info_dict

In [74]:
int(re.findall('task_number([0-9])','P 0 task_number0')[0])

0

##  Loop analysis over all subjects

In [75]:
get_probe_trial_performance_homecage(lines)

({'6_3': [True, True, True, True], '1_6': [True, True], '3_1': [False, True]},
 {'6_3': [True, True, True, True], '1_6': [True, True], '3_1': [False, True]})

In [76]:
lines[-2]

'D 509950 10\n'

In [77]:
eval(re.findall('seq:(\[.*\])', 'P 1 SEQ:[3, 6, 1]\n',)[0])

IndexError: list index out of range

In [79]:
#subject_folders = [r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1\C00494062_5\16190096773636293_task"]

In [80]:
experiment_folders = [r"/Users/yves/Downloads/ec_lineloop_1/",
                     r"/Users/yves/Downloads/ec_lineloop_2/"]

subject_folders = []
for ef in experiment_folders:
    sf1 = os.listdir(ef)
    
    for sf_ in sf1:
        sf = os.path.join(ef,sf_)
        subject_folders.append(os.path.join(sf,os.listdir(sf)[0]))
    

In [101]:
#How will it be easiest to store this data?

df = pd.DataFrame(columns=['subject','exp_name', 'date', 'seqs', 'task_nr', 'probe_dict', 'len_seq'])

In [102]:
n = 30
PTHRESH = 0.05
for ROOT in subject_folders:
    print(ROOT)
    fs = [i for i in os.listdir(ROOT) if '_taskFile' not in i]
    probe_dict = {}
    for f_ in fs:
        if 'follow_the_light_2_graphs' in f_:
            #print(f_)
            fpath = os.path.join(ROOT,f_)
            with open(fpath,'r') as f:
                lines = f.readlines()

            probe_dict,info_dict = get_probe_trial_performance_homecage(lines,probe_dict=probe_dict)
            #print(probe_dict)
            df = df.append(info_dict,ignore_index=True)
    #print(len(probe_dict))
    #for k,v in probe_dict.items():
    #    print(k,np.mean(v),len(v))
    #print()
    print(np.sum([stt.binom_test(sum(v[-n:]),len(v[-n:]),p=1/9.)<PTHRESH for k,v in probe_dict.items()]),
             len([stt.binom_test(sum(v[-n:]),len(v[-n:]),p=1/9.)<PTHRESH for k,v in probe_dict.items()]),
         np.mean([stt.binom_test(sum(v[-n:]),len(v[-n:]),p=1/9.)<PTHRESH for k,v in probe_dict.items()]),
             all([stt.binom_test(sum(v[-n:]),len(v[-n:]),p=1/9.)<PTHRESH for k,v in probe_dict.items()]))
    print()

/Users/yves/Downloads/ec_lineloop_1/C00494062_4/16190096674466636_task
4 10 0.4 False

/Users/yves/Downloads/ec_lineloop_1/C00494062_3/16190096502200105_task
10 10 1.0 True

/Users/yves/Downloads/ec_lineloop_1/C00494062_2/1619009637253015_task
5 12 0.4166666666666667 False

/Users/yves/Downloads/ec_lineloop_1/C00494062_5/16190096773636293_task
8 8 1.0 True

/Users/yves/Downloads/ec_lineloop_1/C00494062_1/16190095855355382_task
7 8 0.875 False

/Users/yves/Downloads/ec_lineloop_2/C00499495_3/16201231194133265_task
6 10 0.6 False

/Users/yves/Downloads/ec_lineloop_2/C00499495_4/1620123129700262_task
4 10 0.4 False

/Users/yves/Downloads/ec_lineloop_2/C00499495_5/1620123140332491_task
6 8 0.75 False

/Users/yves/Downloads/ec_lineloop_2/C00499495_2/16201231071871436_task
0.0 0 nan True

/Users/yves/Downloads/ec_lineloop_2/C00499495_1/16201230948022072_task
5 12 0.4166666666666667 False

/Users/yves/Downloads/ec_lineloop_2/C00499495_6/16201231547824955_task


/Users/yves/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/yves/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


7 12 0.5833333333333334 False



In [103]:
df

,subject,exp_name,date,seqs,task_nr,probe_dict,len_seq
0,C00494062_4,ec_lineloop_1,2021-05-10-222525,"[[3, 6, 1]]",[1],{},[3]
1,C00494062_4,ec_lineloop_1,2021-05-10-214450,"[[3, 6, 1]]",[1],"{'0_5': [False], '4_5': [False], '5_4': [False]}",[3]
2,C00494062_4,ec_lineloop_1,2021-05-11-024020,"[[3, 6, 1]]",[1],{},[3]
3,C00494062_4,ec_lineloop_1,2021-05-10-130945,"[[3, 6, 1]]",[0],"{'1_6': [True], '8_1': [False], '6_1': [False,...",[3]
4,C00494062_4,ec_lineloop_1,2021-05-10-173501,"[[3, 6, 1]]",[1],"{'0_5': [False, False]}",[3]
...,...,...,...,...,...,...,...
386,C00499495_6,ec_lineloop_2,2021-05-07-131309,[],[],{},[]
387,C00499495_6,ec_lineloop_2,2021-05-11-004937,"[[3, 6, 1]]",[0],{},[3]
388,C00499495_6,ec_lineloop_2,2021-05-10-233746,"[[3, 6, 1]]",[0],"{'1_6': [True, True, True, True, True, True], ...",[3]
389,C00499495_6,ec_lineloop_2,2021-05-10-230505,"[[3, 6, 1]]",[0],"{'3_1': [False, True], '1_6': [True], '6_1': [...",[3]


In [105]:
ROOT = r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1/C00494062_1/16190095855355382_task/"

fs = [i for i in os.listdir(ROOT) if '_taskFile' not in i]

In [98]:
#fpath = r'C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_2/C00499495_6/16201231547824955_task/C00499495_6_ec_lineloop_2_follow_the_light_2_graphs'

In [99]:
probe_dict = {}
for f_ in fs:
    if 'follow_the_light_2_graphs' in f_:
        #print(f_)
        fpath = os.path.join(ROOT,f_)
        with open(fpath,'r') as f:
            lines = f.readlines()

        task_lines = []
        for l_ in lines:
            if 'POKED' in l_:
                task_lines.append(l_)
        probe_dict = get_probe_trial_performance_homecage(task_lines,probe_dict=probe_dict)
        #print(probe_dict)

In [150]:
def get_probe_trial_performance_homecage(lines,probe_dict={}):
    """ """
    times = []
    probe_perf = []
    target_hist = []; probe_hist = []; poked_hist = []; dir_hist = []; seq_hist = []
    #if probe_dict is None: 
    #    probe_dict = {}
    for ix,l_ in enumerate(lines):
        
        if 'TARGET' in l_:
            t_ = re.findall('P ([0-9]*) R',l_)[0]
            times.append(float(t_))
            probe = eval(re.findall('PROBE: (False|True)',l_)[0])
            target = int(re.findall('TARGET: ([0-9]*)',l_)[0])
            poked = int(re.findall('POKED: ([0-9]*)',l_)[0])
            direction = int(re.findall('DIR: (.*)',l_)[0])
            seq_ix = int(re.findall('SEQ_IX: ([0-9]*)',l_)[0])

            if probe:
                if not (probe_hist[-1] and (target==target_hist[-1])):
                    correct_poke = poked==target
                    probe_perf.append(correct_poke)
                    transition = str(seq_hist[-1]) + '_' + str(seq_ix)
                    if transition in probe_dict.keys():
                        probe_dict[transition].append(correct_poke)
                    else:
                        probe_dict[transition] = [correct_poke]
            target_hist.append(target); probe_hist.append(probe); dir_hist.append(direction);
            poked_hist.append(poked); seq_hist.append(seq_ix)
    #print(len(probe_perf))
    return probe_dict

In [ ]:
 A B C

In [13]:
l_

'P 146 REWS:0, POKED: 4, PROBE: False , TARGET: 6,SEQ_IX: 1, DIR: 1\n'